In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
# import data
links = pd.read_csv("testlinks.csv")
nodes = pd.read_csv("testnodes.csv")
loads = pd.read_csv("testloads.csv")

In [12]:
# parameters
e = 0.9995 #efficiency per mile

# indices of generating nodes
gen_nodes = [1,4]
gen_costs = [0,1]
gen_lims = [6, 10]

In [22]:
# Create a new model
m = gp.Model("test")

# Generators
g = {}
g[1] = m.addVar(lb = 0, ub = 6)
g[4] = m.addVar(lb = 0, ub = 10)

# Flow
f = {}

f[(1,2)] = m.addVar(lb = 0, ub = 10)
f[(2,1)] = m.addVar(lb = 0, ub = 10)

f[(2,3)] = m.addVar(lb = 0, ub = 5)
f[(3,2)] = m.addVar(lb = 0, ub = 5)

f[(2,4)] = m.addVar(lb = 0, ub = 5)
f[(4,2)] = m.addVar(lb = 0, ub = 5)

f[(3,4)] = m.addVar(lb = 0, ub = 5)
f[(4,3)] = m.addVar(lb = 0, ub = 5)

# Junctions
j = {}

j[1] = m.addConstr(g[1] + f[(2,1)] - f[(1,2)] == 0)
j[2] = m.addConstr(f[(1,2)] - f[(2,1)] + f[(3,2)] - f[(2,3)] + f[(4,2)] - f[(2,4)] == 5)
j[2] = m.addConstr(f[(2,3)] - f[(3,2)] + f[(4,3)] - f[(3,4)] == 7)
j[4] = m.addConstr(g[4] + f[(2,4)] + f[(3,4)] - f[(4,2)] - f[(4,3)] == 0)

# Objective
cost = gp.LinExpr(0)
cost.add(g[4])
cost.add(f[(1,2)], 0.01)
cost.add(f[(2,1)], 0.01)

obj = m.setObjective(cost, GRB.MINIMIZE)

# Optimize
m.optimize()

# Print
print(m.getAttr('x', f))



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 10 columns and 18 nonzeros
Model fingerprint: 0x8388d30a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 7e+00]
Presolve removed 4 rows and 10 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0600000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  6.060000000e+00
{(1, 2): 6.0, (2, 1): 0.0, (2, 3): 2.0, (3, 2): 0.0, (2, 4): 0.0, (4, 2): 1.0, (3, 4): 0.0, (4, 3): 5.0}
